## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,description
0,0,Cape Town,-33.9258,18.4232,65.26,35,0,5.75,ZA,clear sky
1,1,Mahebourg,-20.4081,57.7000,70.09,73,20,9.22,MU,few clouds
2,2,Mataura,-46.1927,168.8643,39.47,91,90,3.67,NZ,overcast clouds
3,3,Pangnirtung,66.1451,-65.7125,46.40,71,90,5.75,CA,overcast clouds
4,4,Katobu,-4.9408,122.5276,71.28,91,19,3.24,ID,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(8)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,description
1,1,Mahebourg,-20.4081,57.7000,70.09,73,20,9.22,MU,few clouds
4,4,Katobu,-4.9408,122.5276,71.28,91,19,3.24,ID,few clouds
8,8,Carauari,-4.8828,-66.8958,87.28,57,93,0.96,BR,overcast clouds
12,12,Hurghada,27.2574,33.8129,89.46,36,0,22.55,EG,clear sky
13,13,Ostersund,63.1792,14.6357,74.61,49,0,5.75,SE,clear sky
16,16,Lengshuijiang,27.6881,111.4294,75.00,95,17,3.20,CN,few clouds
17,17,Jamestown,42.0970,-79.2353,78.39,58,1,5.99,US,clear sky
18,18,Cedar Falls,42.5278,-92.4455,82.22,75,40,1.01,US,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        3
description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
clean_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        0
description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,description,Lat,Lng,Hotel Name
1,Mahebourg,MU,70.09,few clouds,-20.4081,57.7000,
4,Katobu,ID,71.28,few clouds,-4.9408,122.5276,
8,Carauari,BR,87.28,overcast clouds,-4.8828,-66.8958,
12,Hurghada,EG,89.46,clear sky,27.2574,33.8129,
13,Ostersund,SE,74.61,clear sky,63.1792,14.6357,
16,Lengshuijiang,CN,75.00,few clouds,27.6881,111.4294,
17,Jamestown,US,78.39,clear sky,42.0970,-79.2353,
18,Cedar Falls,US,82.22,scattered clouds,42.5278,-92.4455,
20,Georgetown,MY,85.89,few clouds,5.4112,100.3354,
22,Changping,CN,83.52,few clouds,40.2186,116.2197,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]["name"]
    except:
        print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head(10)

,City,Country,Max Temp,description,Lat,Lng,Hotel Name
1,Mahebourg,MU,70.09,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
4,Katobu,ID,71.28,few clouds,-4.9408,122.5276,La Ode Husein
8,Carauari,BR,87.28,overcast clouds,-4.8828,-66.8958,Hotel Fellip
12,Hurghada,EG,89.46,clear sky,27.2574,33.8129,Sea Star Beau Rivage Hotel
13,Ostersund,SE,74.61,clear sky,63.1792,14.6357,Best Western Hotel Gamla Teatern
16,Lengshuijiang,CN,75.00,few clouds,27.6881,111.4294,Sheraton
17,Jamestown,US,78.39,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
18,Cedar Falls,US,82.22,scattered clouds,42.5278,-92.4455,"Country Inn & Suites by Radisson, Cedar Falls, IA"
20,Georgetown,MY,85.89,few clouds,5.4112,100.3354,Cititel Penang
22,Changping,CN,83.52,few clouds,40.2186,116.2197,Huachang Hotel


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
info_box_template

,City,Country,description,Max Temp
1,Mahebourg,MU,few clouds,70.09
4,Katobu,ID,few clouds,71.28
8,Carauari,BR,overcast clouds,87.28
12,Hurghada,EG,clear sky,89.46
13,Ostersund,SE,clear sky,74.61
...,...,...,...,...
685,Hobyo,SO,overcast clouds,79.23
689,Santa Cruz,BO,clear sky,85.46
690,Merke,KZ,clear sky,85.86
691,Peniche,PT,clear sky,78.62


In [53]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations)
# 11b. Display the figure
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))